# Deep Learning for NLP - Creating a chatbot

In [ ]:
!git clone https://github.com/deepanrajm/deep_learning.git

In [ ]:
#Library Imports
import pickle
import numpy as np

In [ ]:
#retrieve training data
with open('deep_learning/Attention_model/train_qa.txt', 'rb') as f:
    train_data = pickle.load(f)

In [ ]:
#retrieve test data
with open('deep_learning/Attention_model/test_qa.txt', 'rb') as f:
    test_data = pickle.load(f)

In [ ]:
#Number of training instances
len(train_data)

In [ ]:
#Number of test instances
len(test_data)

In [ ]:
#Example of one of the instances
train_data[10]

In [ ]:
' '.join(train_data[10][0])

In [ ]:
' '.join(train_data[10][1])

In [ ]:
train_data[10][2]

In [ ]:
#First we will build a set of all the words in the dataset:
vocab = set()
for story, question, answer in train_data:
    vocab = vocab.union(set(story)) #Set returns unique words in the sentence
                                    #Union returns the unique common elements from a two sets
    vocab = vocab.union(set(question))

In [ ]:
vocab.add('no')
vocab.add('yes')

In [ ]:
vocab

In [ ]:
#Calculate len and add 1 for Keras placeholder - Placeholders are used to feed in the data to the network. 
#They need a data type, and have optional shape arguements.
#They will be empty at first, and then the data will get fed into the placeholder
vocab_len = len(vocab) + 1

In [ ]:
vocab_len

In [ ]:
#Now we are going to calculate the longest story and the longest question
#We need this for the Keras pad sequences. 
#Keras training layers expect all of the input to have the same length, so 
#we need to pad 
all_data = test_data + train_data

In [ ]:
all_story_lens = [len(data[0]) for data in all_data]

In [ ]:
max_story_len = (max(all_story_lens))

In [ ]:
max_question_len = max([len(data[1]) for data in all_data])

## Vectorizing the data

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [ ]:
#Create an instance of the tokenizer object:
tokenizer = Tokenizer(filters = [])
tokenizer.fit_on_texts(vocab)

In [ ]:
#Dictionary that maps every word in our vocab to an index
# It has been automatically lowercased
#This tokenizer can give different indexes for different words depending on when we run it
tokenizer.word_index

In [ ]:
#Tokenize the stories, questions and answers:
train_story_text = []
train_question_text = []
train_answers = []

In [ ]:
#Separating each of the elements
for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question) 
    train_answers.append(answer)
    

In [ ]:
#Coverting the text into the indexes 
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [ ]:
#Create a function for vectorizing the stories, questions and answers:
def vectorize_stories(data,word_index = tokenizer.word_index, max_story_len = max_story_len, max_question_len = max_question_len):
    #vectorized stories:
    X = []
    #vectorized questions:
    Xq = []
    #vectorized answers:
    Y = []
    
    for story, question, answer in data:
        #Getting indexes for each word in the story
        x = [word_index[word.lower()] for word in story]
        #Getting indexes for each word in the story
        xq = [word_index[word.lower()] for word in question]
        #For the answers
        y = np.zeros(len(word_index) + 1) #Index 0 Reserved when padding the sequences
        y[word_index[answer]] = 1
        
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    #Now we have to pad these sequences:
    return(pad_sequences(X,maxlen=max_story_len), pad_sequences(Xq, maxlen=max_question_len), np.array(Y))
        

In [ ]:
inputs_train, questions_train, answers_train = vectorize_stories(train_data)

In [ ]:
inputs_test, questions_test, answers_test = vectorize_stories(test_data)

In [ ]:
inputs_train[0]

In [ ]:
train_story_text[0]

In [ ]:
train_story_seq[0]

## Building the Network

In [ ]:
#Imports
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate, LSTM

In [ ]:
# We need to create the placeholders 
#The Input function is used to create a keras tensor
#PLACEHOLDER shape = (max_story_len,batch_size)
#These are our placeholder for the inputs, ready to recieve batches of the stories and the questions
input_sequence = Input((max_story_len,)) #As we dont know batch size yet
question = Input((max_question_len,))

In [ ]:
#Create input encoder M:
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_len,output_dim = 64)) #From paper
input_encoder_m.add(Dropout(0.3))

#Outputs: (Samples, story_maxlen,embedding_dim) -- Gives a list of the lenght of the samples where each item has the
#lenght of the max story lenght and every word is embedded in the embbeding dimension

In [ ]:
#Create input encoder C:
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_len,output_dim = max_question_len)) #From paper
input_encoder_c.add(Dropout(0.3))

#Outputs: (samples, story_maxlen, max_question_len)

In [ ]:
#Create question encoder:
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_len,output_dim = 64,input_length=max_question_len)) #From paper
question_encoder.add(Dropout(0.3))

#Outputs: (samples, question_maxlen, embedding_dim)

In [ ]:
#Now lets encode the sequences, passing the placeholders into our encoders:
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [ ]:
#Use dot product to compute similarity between input encoded m and question 
#Like in the paper:
match = dot([input_encoded_m,question_encoded], axes = (2,2))
match = Activation('softmax')(match)

In [ ]:
#For the response we want to add this match with the ouput of input_encoded_c
response = add([match,input_encoded_c])
response = Permute((2,1))(response) #Permute Layer: permutes dimensions of input

In [ ]:
#Once we have the response we can concatenate it with the question encoded:
answer = concatenate([response, question_encoded])

In [ ]:
# Reduce the answer tensor with a RNN (LSTM)
answer = LSTM(32)(answer)

In [ ]:
#Regularization with dropout:
answer = Dropout(0.5)(answer)
#Output layer:
answer = Dense(vocab_len)(answer) #Output shape: (Samples, Vocab_size) #Yes or no and all 0s

In [ ]:
#Now we need to output a probability distribution for the vocab, using softmax:
answer = Activation('softmax')(answer)

In [ ]:
#Now we build the final model:
model = Model([input_sequence,question], answer)

In [ ]:
model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
#Categorical instead of binary cross entropy as because of the way we are training
#we could actually see any of the words from the vocab as output
#however, we should only see yes or no

## Training and testing the model

In [ ]:
history = model.fit([inputs_train,questions_train],answers_train, batch_size = 32, epochs = 10, validation_data = ([inputs_test,questions_test],answers_test))

In [ ]:
filename = 'Z_chatbot_100_epochs.h5'
model.save(filename)

In [ ]:
#To load a model that we have already trained and saved:
model.load_weights('Z_chatbot_100_epochs.h5')

In [ ]:
#Lets check out the predictions on the test set:
#These are just probabilities for every single word on the vocab
pred_results = model.predict(([inputs_test,questions_test]))

In [ ]:
#First test data point
test_data[0]

In [ ]:
#These are the probabilities for the vocab words using the 1st sentence
pred_results[0]

In [ ]:
val_max = np.argmax(pred_results[0])

In [ ]:
for key,val in tokenizer.word_index.items():
    if val == val_max:
        k = key
print(k)

In [ ]:
#See probability:
pred_results[0][val_max]

In [ ]:
#Now, we can make our own questions using the vocabulary we have
vocab

In [ ]:
my_story = 'Sandra picked up the milk . Mary travelled left . '

In [ ]:
my_story.split()

In [ ]:
my_question = 'Sandra got the milk ?'

In [ ]:
my_question.split()

In [ ]:
#Put the data in the same format as before
my_data = [(my_story.split(), my_question.split(),'yes')]

In [ ]:
#Vectorize this data
my_story, my_ques, my_ans = vectorize_stories(my_data)

In [ ]:
#Make the prediction
pred_results = model.predict(([my_story,my_ques]))

In [ ]:
val_max = np.argmax(pred_results[0])

In [ ]:
#Correct prediction!
for key,val in tokenizer.word_index.items():
    if val == val_max:
        k = key
print(k)

In [ ]:
#Confidence
pred_results[0][val_max]